In [1]:
# Parameters
cpu = 8
group_name = "DG_Glut"
mem_gb = 10


In [2]:
import pandas as pd
from scipy.stats import norm, zscore
import numpy as np
import seaborn as sns
import subprocess
import matplotlib.pyplot as plt 

In [3]:
#group_name = 'DG_Glut'

In [4]:
home_dir = '/home/qzeng_salk_edu/project/241018_pre_ml'
ct = group_name

In [5]:
loop_dir = '/data/female-amb/Diff.Result/loop_ct_age/diff'

In [6]:
loopq = pd.read_hdf(f'{loop_dir}/{ct}/loop_Q.hdf', key='data')
loopt = pd.read_hdf(f'{loop_dir}/{ct}/loop_T.hdf', key='data')

loopq.columns = [f"{ct}.2mo.Q", f"{ct}.9mo.Q", f"{ct}.18mo.Q"]
loopt.columns = [f"{ct}.2mo.T", f"{ct}.9mo.T", f"{ct}.18mo.T"]

In [7]:
loopall = pd.read_hdf(f"{loop_dir}/{ct}/merged_loop.hdf")
loopall = pd.concat([loopall, loopq, loopt], axis = 1)
loopall.head()

,0,1,2,3,4,5,Qanova,Eanova,Tanova,DG_Glut.2mo.Q,DG_Glut.9mo.Q,DG_Glut.18mo.Q,DG_Glut.2mo.T,DG_Glut.9mo.T,DG_Glut.18mo.T
0,chr1,3080000,3090000,chr1,3660000,3670000,4.272670,22.574261,2.617610,0.002639,0.001774,0.002675,0.122697,0.037139,0.106146
1,chr1,3080000,3090000,chr1,3670000,3680000,4.306555,22.650797,2.116224,0.002834,0.001820,0.002545,0.122018,0.048540,0.117363
2,chr1,3080000,3090000,chr1,4130000,4140000,2.975958,14.647219,2.281174,0.002234,0.002000,0.002914,0.147600,0.061390,0.086619
3,chr1,3090000,3100000,chr1,3650000,3660000,3.583169,31.817041,5.230794,0.002580,0.001863,0.002793,0.101614,0.016242,0.138994
4,chr1,3090000,3100000,chr1,3660000,3670000,2.078557,30.985931,3.964074,0.002834,0.002098,0.002659,0.127612,0.064369,0.175700


In [8]:
sell = ((zscore(np.log(loopall['Qanova']))>norm.isf(0.2)) & 
        (zscore(np.log(loopall['Tanova']))>norm.isf(0.2)) )
sell.sum()

60145

In [9]:
loopall['Diff_Loop'] = 0
loopall.loc[sell, 'Diff_Loop'] = 1
loopall.to_csv(f"{ct}.anchor1_loop.bed", sep = '\t', header = False, index = False)

In [10]:
# move columns 3 4 5 in to the first three
cols = loopall.columns.tolist()
cols = cols[3:6] + cols[:3] + cols[6:] 
loopall = loopall[cols]
loopall.to_csv(f"{ct}.anchor2_loop.bed", sep = '\t', header = False, index = False)

In [11]:
# so diff loop as two anchors, so two chro:start-end region, i want to get the intersection with another bed file as long as it intersect with eithe ranhocr
command = f"bedtools intersect -a {ct}.anchor1_loop.bed -b {home_dir}/mm10_gene_2kb.bed -wa -wb > {ct}.anchor1_loop_gene.bed"
subprocess.run(command, shell=True)

command = f"bedtools intersect -a {ct}.anchor2_loop.bed -b {home_dir}/mm10_gene_2kb.bed -wa -wb > {ct}.anchor2_loop_gene.bed"
subprocess.run(command, shell=True)

CompletedProcess(args='bedtools intersect -a DG_Glut.anchor2_loop.bed -b /home/qzeng_salk_edu/project/241018_pre_ml/mm10_gene_2kb.bed -wa -wb > DG_Glut.anchor2_loop_gene.bed', returncode=0)

In [12]:
subprocess.run(f"rm {ct}.anchor1_loop.bed", shell=True)
subprocess.run(f"rm {ct}.anchor2_loop.bed", shell=True)

CompletedProcess(args='rm DG_Glut.anchor2_loop.bed', returncode=0)

In [13]:
# load the output files and concat
anchor1 = pd.read_csv(f"{ct}.anchor1_loop_gene.bed", sep = '\t', header = None)
anchor2 = pd.read_csv(f"{ct}.anchor2_loop_gene.bed", sep = '\t', header = None)
# concat
anchor = pd.concat([anchor1, anchor2], axis = 0)

In [14]:
anchor.columns = ['chrom','anchor1_start','anchor1_end','chrom','anchor2_start','anchor2_end',
                'Qanova','Eanova','Tanova','2mo.Q','9mo.Q','18mo.Q','2mo.T','9mo.T','18mo.T','Diff_Loop',
                'gene_chrom','gene_start','gene_end','gene_id','strand','gene_name','gene_type']
anchor

,chrom,anchor1_start,anchor1_end,chrom,anchor2_start,anchor2_end,Qanova,Eanova,Tanova,2mo.Q,...,9mo.T,18mo.T,Diff_Loop,gene_chrom,gene_start,gene_end,gene_id,strand,gene_name,gene_type
0,chr1,3100000,3110000,chr1,3400000,3410000,0.635926,34.182580,2.416174,0.002807,...,0.044727,0.119007,0,chr1,3100016,3104125,ENSMUSG00000064842.1,+,Gm26206,snRNA
1,chr1,3100000,3110000,chr1,3420000,3430000,2.407307,27.711254,1.082402,0.003030,...,0.059812,0.111905,0,chr1,3100016,3104125,ENSMUSG00000064842.1,+,Gm26206,snRNA
2,chr1,3100000,3110000,chr1,3430000,3440000,3.186451,31.353325,2.305675,0.002938,...,0.051179,0.127769,0,chr1,3100016,3104125,ENSMUSG00000064842.1,+,Gm26206,snRNA
3,chr1,3100000,3110000,chr1,3440000,3450000,5.206938,29.662554,1.694190,0.002779,...,0.075505,0.144651,0,chr1,3100016,3104125,ENSMUSG00000064842.1,+,Gm26206,snRNA
4,chr1,3100000,3110000,chr1,3450000,3460000,3.630835,28.548758,0.993784,0.002604,...,0.102928,0.143655,0,chr1,3100016,3104125,ENSMUSG00000064842.1,+,Gm26206,snRNA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606505,chr19,60550000,60560000,chr19,60480000,60490000,3.364248,9.657704,1.076062,0.008261,...,0.050714,0.099452,0,chr19,60522696,60583023,ENSMUSG00000033417.11,-,Cacul1,protein_coding
606506,chr19,60750000,60760000,chr19,60580000,60590000,6.412291,22.808350,4.730181,0.006420,...,0.109658,0.220566,1,chr19,60753987,60761913,ENSMUSG00000072437.3,+,Nanos1,protein_coding
606507,chr19,60750000,60760000,chr19,60580000,60590000,6.412291,22.808350,4.730181,0.006420,...,0.109658,0.220566,1,chr19,60759116,60792693,ENSMUSG00000024991.7,-,Eif3a,protein_coding
606508,chr19,61070000,61080000,chr19,61010000,61020000,2.321018,7.665514,2.223011,0.010920,...,0.137818,0.151214,0,chr19,60887749,61094553,ENSMUSG00000003228.8,+,Grk5,protein_coding


In [15]:
subprocess.run(f"rm {ct}.anchor1_loop_gene.bed", shell=True)
subprocess.run(f"rm {ct}.anchor2_loop_gene.bed", shell=True)

CompletedProcess(args='rm DG_Glut.anchor2_loop_gene.bed', returncode=0)

In [16]:
# add a distance columns, anchor1_start to gene_start and gene end, use which ever is closer
anchor['anchor1_distance'] = np.minimum(abs(anchor['anchor1_start'] - anchor['gene_start']), abs(anchor['anchor1_start'] - anchor['gene_end']))
anchor['anchor2_distance'] = np.minimum(abs(anchor['anchor2_start'] - anchor['gene_start']), abs(anchor['anchor2_start'] - anchor['gene_end']))
anchor['gene_length'] = anchor['gene_end'] - anchor['gene_start']
# drop chrom	anchor1_start	anchor1_end	chrom	anchor2_start	anchor2_end	, 'gene_chrom', 'gene_start','gene_end', 'strand', 'gene_type'
anchor = anchor.drop(columns = ['chrom','anchor1_start','anchor1_end','chrom','anchor2_start','anchor2_end','gene_chrom', 'gene_start','gene_end', 'strand', 'gene_type'])
anchor

,Qanova,Eanova,Tanova,2mo.Q,9mo.Q,18mo.Q,2mo.T,9mo.T,18mo.T,Diff_Loop,gene_id,gene_name,anchor1_distance,anchor2_distance,gene_length
0,0.635926,34.182580,2.416174,0.002807,0.002384,0.002618,0.053451,0.044727,0.119007,0,ENSMUSG00000064842.1,Gm26206,16,295875,4109
1,2.407307,27.711254,1.082402,0.003030,0.002189,0.002847,0.097521,0.059812,0.111905,0,ENSMUSG00000064842.1,Gm26206,16,315875,4109
2,3.186451,31.353325,2.305675,0.002938,0.002243,0.003327,0.105777,0.051179,0.127769,0,ENSMUSG00000064842.1,Gm26206,16,325875,4109
3,5.206938,29.662554,1.694190,0.002779,0.002293,0.003597,0.115799,0.075505,0.144651,0,ENSMUSG00000064842.1,Gm26206,16,335875,4109
4,3.630835,28.548758,0.993784,0.002604,0.002375,0.003405,0.093010,0.102928,0.143655,0,ENSMUSG00000064842.1,Gm26206,16,345875,4109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606505,3.364248,9.657704,1.076062,0.008261,0.006708,0.008343,0.097121,0.050714,0.099452,0,ENSMUSG00000033417.11,Cacul1,27304,42696,60327
606506,6.412291,22.808350,4.730181,0.006420,0.004742,0.007094,0.128506,0.109658,0.220566,1,ENSMUSG00000072437.3,Nanos1,3987,173987,7926
606507,6.412291,22.808350,4.730181,0.006420,0.004742,0.007094,0.128506,0.109658,0.220566,1,ENSMUSG00000024991.7,Eif3a,9116,179116,33577
606508,2.321018,7.665514,2.223011,0.010920,0.009670,0.011467,0.216247,0.137818,0.151214,0,ENSMUSG00000003228.8,Grk5,24553,84553,206804


In [17]:
anchor.to_csv(f"{home_dir}/ml_input/{ct}/{ct}.Loop_gene.csv",  header = True, index = False)

In [18]:
import subprocess
#ct = 'L6_IT_CTX_Glut'
#subprocess.run(f"tar -cjvf ml_input/{ct}/{ct}.Loop_gene.csv.gz ml_input/{ct}/{ct}.Loop_gene.csv", shell=True)
subprocess.run(f"gzip -k {home_dir}/ml_input/{ct}/{ct}.Loop_gene.csv", shell=True)

CompletedProcess(args='gzip -k /home/qzeng_salk_edu/project/241018_pre_ml/ml_input/DG_Glut/DG_Glut.Loop_gene.csv', returncode=0)

In [19]:
subprocess.run(f"rm {home_dir}/ml_input/{ct}/{ct}.Loop_gene.csv", shell=True)

CompletedProcess(args='rm /home/qzeng_salk_edu/project/241018_pre_ml/ml_input/DG_Glut/DG_Glut.Loop_gene.csv', returncode=0)

In [20]:
#anchor.to_csv(f"ml_input/{ct}.diff_loop_gene.csv",  header = True, index = False)